In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [7]:
mol_type = "6r7m"
algo = "rwm"
energy = "ma"
folder = "../../Data/hpc_out/scans/2_6r7m/1/"
#folder = "../../Data/hpc_out/$(algo)_$(energy)_2_$(mol_type)/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
                println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                continue
            end

            # if input["overlap_slope"] != 1.3
            #     continue
            # end

            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(input["overlap_slope"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")
for elem in evaluation_strings
    println(elem)
end

Minimal configuration id: 11 | Average acceptance rate: 0.5530404458075519
1: 4.0 | 1.0 | 0.6509601 | 8762.741530448666, 8693.493355916318, 8762.741530522613 | 11.611596755552169
2: 4.0 | 1.0 | 0.86130106 | 8762.74153029157, 8703.909754517545, 8762.74153040259 | 15.318596976859919
3: 4.0 | 1.0 | 0.28527653 | 8762.741530466392, 8690.922405680982, 8709.470536500303 | 6.639948964717787
4: 4.0 | 1.0 | 0.36895216 | 8762.741530328369, 8685.385762741133, 8762.741530406198 | 6.059323040554525
5: 4.0 | 1.0 | 0.43615705 | 8762.741530297966, 8689.702945558924, 8698.690042092383 | 20.95872333015336
6: 4.0 | 1.0 | 0.7341769 | 8762.74153042557, 8714.380068141427, 8729.21517592163 | 17.594520856115043
7: 4.0 | 1.0 | 0.2671662 | 8762.74153031107, 8694.127865659048, 8712.375718027304 | 6.376366248362985
8: 4.0 | 1.0 | 0.36303243 | 8762.741530428488, 8698.27185510424, 8762.741530109533 | 21.444639495686086
9: 4.0 | 1.0 | 0.72187555 | 8762.741530082649, 8703.489534731349, 8762.741530310326 | 22.231351677

In [4]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [ ]:
@load "$(folder)/11_$(algo_e)_2_$(mol_type).jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [15]:
"$(folder)/11_$(algo_e)_2_$(mol_type).jld2"

"../../Data/hpc_out/2_6r7m/rwm_ma_2_6r7m//11_rwm_ma_2_6r7m.jld2"

In [13]:
using GeometryBasics

In [ ]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
